<a href="https://colab.research.google.com/github/bulentsoykan/CVRP/blob/main/CVRP_Approaches_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing 03 approaches for a CVRP problem
## a). Dantzig, Fulkerson, Jhonson approach (Eliminating subtours via subset constraints)
## b). Miller-Tucker-Zemlin (MTZ) approach
## c). Lazy constraints approach

## Importing Packages

In [27]:
%pip install -qq gurobipy

In [28]:
from google.colab import userdata
LICENSEID = int(userdata.get('LICENSEID'))
WLSACCESSID = userdata.get('WLSACCESSID')
WLSSECRET = userdata.get('WLSSECRET')

In [29]:
import gurobipy as gp
options = {
    "WLSACCESSID": WLSACCESSID,
    "WLSSECRET": WLSSECRET,
    "LICENSEID": LICENSEID,
}

In [30]:
env= gp.Env(params=options)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2514190
Academic license 2514190 - for non-commercial use only - registered to bu___@ucf.edu


In [31]:
from datetime import datetime as dt, timedelta as td
from folium.plugins import BeautifyIcon
from tqdm import tqdm

# Pure Imports

import networkx as nx
import pandas as pd
import numpy as np
import itertools
import random
import folium
import math
import time
import os

# Gurobi Imports

from gurobipy import GRB
import gurobipy as gp

## User Inputs

In [32]:
# Data input config

daughter_nodes_to_consider = 10
cap_per_vehicle = 1000
methods_to_try = ['a', 'b', 'c']  # 'a', 'b' or 'c'

## Utility Functions

In [33]:
def haversine(lon1: float, lat1: float, lon2: float, lat2: float) -> float:
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a = math.sin(d_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return round(c * r, 5)


def findsubsets(size_):
    return list(itertools.combinations(dau_nodes_dict.keys(), size_))


## Random Inputs Generation

In [34]:
# Generating inputs using random

mother_nodes_dict = {f"mot_0": (22.564566, 88.39582)} # Mother node geocodes generation
dau_nodes_dict = {f"dau_{in_}": (random.uniform(21.778553, 27.117798), random.uniform(85.97712, 89.82857))
                       for in_ in range(1, daughter_nodes_to_consider + 1)} # Daughter node geocodes generation
dau_nodes_demand = {f"dau_{in_}": random.randint(int(round(cap_per_vehicle / 4, 0)), int(round(cap_per_vehicle / 1.5, 0)))
                    for in_ in range(1, daughter_nodes_to_consider + 1)}
all_nodes_info_dict = {**dau_nodes_dict, **mother_nodes_dict}
number_of_vehicles = int(np.ceil(sum([dau_nodes_demand[dau_] for dau_ in dau_nodes_dict]) / cap_per_vehicle)) * 2
required_pairs = list(itertools.product(dau_nodes_dict, dau_nodes_dict)) + list(
    itertools.product(dau_nodes_dict, mother_nodes_dict)) + list(
    itertools.product(mother_nodes_dict, dau_nodes_dict)) + list(
    itertools.product(mother_nodes_dict, mother_nodes_dict))

distance_matrix = {pair_ : haversine(lon1=all_nodes_info_dict[pair_[0]][1], lat1=all_nodes_info_dict[pair_[0]][0],
                                     lon2=all_nodes_info_dict[pair_[1]][1], lat2=all_nodes_info_dict[pair_[1]][0])
                   for pair_ in tqdm(required_pairs)}

# Number of Daughter nodes, total demand and number of vehicles

print("**************************************************")
print("Number of Daughter nodes : ", len(dau_nodes_dict))
print("Total demand : ", sum(dau_nodes_demand.values()))
print("Total number of vehicles : ", number_of_vehicles)
print("Total available capacity : ", number_of_vehicles * cap_per_vehicle)
print("**************************************************")

100%|██████████| 121/121 [00:00<00:00, 148830.14it/s]

**************************************************
Number of Daughter nodes :  10
Total demand :  4715
Total number of vehicles :  10
Total available capacity :  10000
**************************************************


## Base Map Creation

In [35]:
# Plot base map

colors_ = ["#000000", "#36454F", "#555555", "#191970", "#000080",
           "#2F4F4F", "#228B22", "#556B2F", "#8B0000", "#800000",
           "#654321", "#FF8C00", "#8A3324", "#800080", "#4B0082",
           "#483D8B", "#800020", "#008080", "#D2691E", "#008B8B"]

lat_list = [all_nodes_info_dict[dau_][0] for dau_ in dau_nodes_dict]
long_list = [all_nodes_info_dict[dau_][1] for dau_ in dau_nodes_dict]
mean_x, mean_y = sum(lat_list) / len(lat_list), sum(long_list) / len(long_list)
map_osm = folium.Map(location=[mean_x, mean_y], zoom_start=10, prefer_canvas=True)
original_mapping_layer = folium.FeatureGroup(name='Original Mapping', show=False).add_to(map_osm)
if 'a' in methods_to_try:
    routing_layer_a = folium.FeatureGroup(name=f'Routing Method-A', show=False).add_to(map_osm)
if 'b' in methods_to_try:
    routing_layer_b = folium.FeatureGroup(name=f'Routing Method-B', show=False).add_to(map_osm)
if 'c' in methods_to_try:
    routing_layer_c = folium.FeatureGroup(name=f'Routing Method-C', show=False).add_to(map_osm)

# Plotting mother nodes

for mother_node in mother_nodes_dict:
    folium.Marker([mother_nodes_dict[mother_node][0], mother_nodes_dict[mother_node][1]],
        popup= f"{mother_node}",
        icon=BeautifyIcon(
        icon='star',
        inner_icon_style='color:black; font-size:30px;',
        background_color='transparent',
        border_color='transparent')).add_to(map_osm)

# Plotting child nodes

for d_node in dau_nodes_dict:
    folium.Circle(location=[all_nodes_info_dict[d_node][0], all_nodes_info_dict[d_node][1]], popup=f"{d_node}",
                  radius=1000, color='red', fill=True,
              opacity=0.9, fill_color='red', stroke=True, weight=3.0).add_to(map_osm)
map_osm.fit_bounds(map_osm.get_bounds(), padding=(30, 30))
folium.LayerControl().add_to(map_osm)

# a). Dantzig, Fulkerson, Jhonson approach

## Solver inputs

In [36]:
time_limit_a = 600
mip_gap_a = 0.01
subtour_bool = True

## Dantzig, Fulkerson, Jhonson approach mathematical formulation

### $ Sets \ : $
#### $ d \ \in \ D: \ Daughter \ Nodes$
#### $ m \ \in \ M: \ Mother \ Nodes$
#### $ k \ \in \ K: \ Total \ available \ vehicles$

### $Parameters:$
#### $ D_{i, \ j}  : \ Distance \ from \ node \ i \ to \ j$
#### $ V_{j} : Demand \ of \ daughter \ node \ j$
#### $ VehCap  : \ Max \ Capacity \ of \ each \ Vehicle$

### $Variables:$
#### $ x_{i, \ j, \ k}  : \ Edge \ selection \ binary \ variable \ between \ two \ daughter \ nodes \ (i \ != \ j) \ via \ vehicle \ k$  
#### $ x_{m, \ i, \ k}  : \ Edge \ selection \ binary \ variable \ between \ mother \ to \ daughter \ node \ via \ vehicle \ k$  
#### $ x_{j, \ m, \ k}  : \ Edge \ selection \ binary \ variable \ between \ daughter \ to \ mother \ node \ via \ vehicle \ k$

### $ Objective \ Function \ : $
### $ min \ \sum_{k\in K} \sum_{m\in M} \sum_{i\in D} \ x_{m, i, k}D_{m, \ i} +
\sum_{k\in K} \sum_{i\in D} \sum_{j\in D} \ x_{i, j, k}D_{i, \ j} +
\sum_{k\in K} \sum_{j\in D} \sum_{m\in M} \ x_{j, m, k}D_{j, \ m}$

### $ Subject \ to \ Constraints: $
#### $1). \ Only \ one \ edge \ entering \ a \ daughter \ node \ can \ be \ active \ (it \ can \ be \ from \ a \ mother \  node \ or \ another \ daughter \ node) $
#### $ \sum_{k\in K} \sum_{m\in M} \ x_{m, i, k} \ + \ \sum_{k\in K} \sum_{j\in D} \ x_{j, i, k} \ = \ 1 \ \forall \ i \in D \$

#### $2). \ At \ any \ node, \ the \ vehicle \ leaving \ should \ be \ the \ same \ as \ vehicle \ entering$
#### $ \ \sum_{m\in M} \ x_{m, i, k} \  + \ \sum_{j\in D} \ x_{j, i, k} \ = \ \sum_{m\in M} \ x_{i, m, k} \  + \ \sum_{j\in D} \ x_{i, j, k} \ \forall \ i \in D \ \forall \ k \in K  $

#### $ \ \sum_{i\in D} \ x_{m, i, k} \ = \ \sum_{j\in D} \ x_{j, m, k} \ \forall \ m \in M \ \forall \ k \in K  $

#### $3). \ If \ a \ vehicle \ is \ being \ used \ to \ service \ atleast \ one \ daughter \ node, \ that \ vehicle \ should \ depart \ from \ a \ mother \ node $

#### $ \ \sum_{i\in D} \ \sum_{j\in D} \ x_{i, j, k} \ <= \ \sum_{m\in M} \ \sum_{j\in D} \ x_{m, j, k} \ * \ M \ \forall \ k \in K  $

#### $ \ \sum_{i\in D} \ \sum_{j\in D} \ x_{i, j, k} \ <= \ z_{k} \ * \ M \ \forall \ k \in K  $

#### $4). A \ vehicle \ can \ depart \ from \ a \ mother \ node \ atmost \ once $

#### $ \ \sum_{m\in M} \ \sum_{j\in D} \ x_{m, j, k} \ <= 1 \ \forall \ k \in K  $

#### $5). \ Each \ vehicle's \ capacity \ constraint $
#### $ \ \sum_{i\in D} \ \sum_{j\in D} \ x_{i, j, k}*V_{j} \ <= \ VehCap \ \forall \ k \in K  $


#### $6). \ Sub-tour \ elimination \ constraint $
#### $ \ \sum_{i \in B} \ \sum_{j \in B} \ x_{i, j, k} \ <= \  |B| \ - \ 1 \ \forall \ B \in \ (D, \ D)\ Edges \ \forall \ k \in K \ , \ |B| \ >= \ 2 $

In [39]:
if 'a' in methods_to_try:
    # Get all possible subsets
    tempo_formulation_a_initial = time.time()
    subtour_possibilities= []
    if subtour_bool:
        for i in tqdm(range(2, len(dau_nodes_dict) + 1)):
            subtour_possibilities += findsubsets(i)
    print("Number of possible sub-tours: ", len(subtour_possibilities))
    # Creating Model

    m_a = gp.Model(env=env)
    m_a.Params.TIME_LIMIT = time_limit_a
    m_a.Params.MIPGap = mip_gap_a

    # Adding Variables

    x_m_i_k_a = m_a.addVars([(mot, dau, k) for dau in dau_nodes_dict.keys()
                             for mot in mother_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)],
                            vtype=GRB.BINARY, name="mot_to_dau")

    x_i_j_k_a = m_a.addVars([(dau_1, dau_2, k) for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1) if dau_1 != dau_2],
                            vtype=GRB.BINARY, name="dau_to_dau")

    x_j_m_k_a = m_a.addVars([(dau, mot, k) for dau in dau_nodes_dict.keys()
                             for mot in mother_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)],
                            vtype=GRB.BINARY, name="dau_to_mot")

    # Adding Objective Function

    mother_to_daughter_dist_sum = [x_m_i_k_a[mot, dau, k] * distance_matrix[mot,dau] for mot in mother_nodes_dict.keys()
                                           for dau in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)]
    daughter_to_daughter_dist_sum = [x_i_j_k_a[dau_1, dau_2, k] * distance_matrix[dau_1, dau_2] for dau_1 in dau_nodes_dict.keys()
                                           for dau_2 in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1) if dau_1 != dau_2]
    daughter_to_mother_dist_sum = [x_j_m_k_a[dau, mot, k] * distance_matrix[dau, mot] for mot in mother_nodes_dict.keys()
                                           for dau in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)]

    m_a.setObjective(gp.quicksum(mother_to_daughter_dist_sum) + gp.quicksum(daughter_to_daughter_dist_sum)
                       + gp.quicksum(daughter_to_mother_dist_sum), sense=GRB.MINIMIZE)

    # Adding Constraints

    # 1). Only one edge entering a daughter node can be active

    for dau_ent in dau_nodes_dict.keys():
        m_a.addConstr(sum([x_i_j_k_a[dau_1, dau_ent, veh_] for dau_1 in dau_nodes_dict.keys()
                                  for veh_ in range(1, number_of_vehicles + 1) if dau_1 != dau_ent]) + sum(
            [x_m_i_k_a[mot_, dau_ent, veh_] for mot_ in mother_nodes_dict.keys()
             for veh_ in range(1, number_of_vehicles + 1)]) == 1, name=f'dau_enter_edge_active_{dau_ent}')


    # 2). At any node, the vehicle leaving should be the same as vehicle entering

    for dau_ in dau_nodes_dict.keys():
        for veh_ in range(1, number_of_vehicles + 1):
            entering_sum_ = sum([x_i_j_k_a[dau_2, dau_, veh_] for dau_2 in dau_nodes_dict.keys() if dau_ != dau_2]) + sum(
                [x_m_i_k_a[mot_, dau_, veh_] for mot_ in mother_nodes_dict.keys()])
            leaving_sum_ = sum(
                [x_i_j_k_a[dau_, dau_2, veh_] for dau_2 in dau_nodes_dict.keys() if dau_ != dau_2]) + sum(
                [x_j_m_k_a[dau_, mot_, veh_] for mot_ in mother_nodes_dict.keys()])

            m_a.addConstr(entering_sum_ == leaving_sum_ , name=f'veh_dau_enter_leave_{dau_}_{veh_}')

    for mot_ in mother_nodes_dict.keys():
        for veh_ in range(1, number_of_vehicles + 1):
            leaving_sum_ = sum([x_m_i_k_a[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()])
            entering_sum_ = sum([x_j_m_k_a[dau_, mot_, veh_] for dau_ in dau_nodes_dict.keys()])
            m_a.addConstr(entering_sum_ == leaving_sum_, name=f'veh_mot_enter_leave_{mot_}_{veh_}')


    # 3). Every vehicle serving atleast one daughter node has to depart from a mother node

    for veh_ in range(1, number_of_vehicles + 1):
        _entering_sum = sum([x_i_j_k_a[dau_1, dau_2, veh_] for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() if dau_1 != dau_2])
        _leaving_sum = sum([x_m_i_k_a[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()
                            for mot_ in mother_nodes_dict.keys()])
        m_a.addConstr(_entering_sum <= _leaving_sum * 10e3, name=f'vehicle_usage_active_{mot_}')


    # 4). Every vehicle can depart from mother node atmost once

    for veh_ in range(1, number_of_vehicles + 1):
        _leaving_sum = sum([x_m_i_k_a[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()
                            for mot_ in mother_nodes_dict.keys()])
        m_a.addConstr(_leaving_sum <= 1, name=f'veh_dept_only_once_{veh_}')


    # 5). Vehicle capacity constraint

    for veh_ in range(1, number_of_vehicles + 1):
        m_a.addConstr(sum([x_i_j_k_a[dau_1, dau_2, veh_] * dau_nodes_demand[dau_2] for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() if dau_1 != dau_2]) + sum(
            [x_m_i_k_a[mot_, dau_2, veh_] * dau_nodes_demand[dau_2] for mot_ in mother_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys()]) <= cap_per_vehicle, name=f'veh_{veh_}_cap')

    # 6). Subtour elimination constraint

    if subtour_bool:
        for subet_ in subtour_possibilities:
            for veh_ in range(1, number_of_vehicles + 1):
                lhs_ = sum([x_i_j_k_a[dau_1, dau_2, veh_] for dau_1 in subet_
                                 for dau_2 in subet_ if dau_1 != dau_2])
                m_a.addConstr(lhs_ <= len(subet_) - 1, name=f'subtour_{"-".join(subet_)}')

    tempo_solving_a_initial = time.time()
    m_a.optimize()
    objective_a = round(m_a.ObjVal, 2)
    number_of_constraints_a = m_a.numconstrs
    formulation_time_a = round((tempo_solving_a_initial - tempo_formulation_a_initial), 2)
    solving_time_a = round((time.time() - tempo_solving_a_initial), 2)
    print("**************************************************")
    print("Total number of constraints: ", m_a.numconstrs)
    print("Formulation time in seconds for method 'a': ", formulation_time_a)
    print("Solving time in seconds for method 'a':", solving_time_a)
    print("**************************************************")

100%|██████████| 9/9 [00:00<00:00, 35916.97it/s]

Number of possible sub-tours:  1013
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2514190 - for non-commercial use only - registered to bu___@ucf.edu
Optimize a model with 10280 rows, 1100 columns and 235700 nonzeros
Model fingerprint: 0xcf2d23aa
Variable types: 0 continuous, 1100 integer (1100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 5604.1082100
Presolve removed 1700 rows and 270 columns
Presolve time: 1.23s
Presolved: 8580 rows, 830 columns, 151860 nonzeros
Variable types: 0 continuous, 830 integer (820 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 2.057830e+03, 175 iterations, 0.10 seconds (0.06 w

## Get results

In [40]:
# Vehicle usage details

if 'a' in methods_to_try:
    veh_cap_used = {veh_: 0 for veh_ in range(1, number_of_vehicles + 1)}
    dau_nodes_visited = []
    for veh_ in range(1, number_of_vehicles + 1):
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if x_m_i_k_a[node_1, node_2, veh_].X > 0.9:
                    veh_cap_used[veh_] += x_m_i_k_a[node_1, node_2, veh_].X * dau_nodes_demand[node_2]
                    dau_nodes_visited.extend([node_1, node_2])

        for node_1 in dau_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2 and x_i_j_k_a[node_1, node_2, veh_].X > 0.9:
                    veh_cap_used[veh_] += x_i_j_k_a[node_1, node_2, veh_].X * dau_nodes_demand[node_2]
                    dau_nodes_visited.extend([node_1, node_2])

        nodes_visited = set(dau_nodes_visited)
        veh_df = pd.DataFrame({'Vehicle_ID': f'Veh_{k}', 'Volume Captured': int(v),
                               'Vehicle Capacity': cap_per_vehicle} for k, v in veh_cap_used.items())
else:
    veh_df = pd.DataFrame()
veh_df

,Vehicle_ID,Volume Captured,Vehicle Capacity
0,Veh_1,0,1000
1,Veh_2,998,1000
2,Veh_3,975,1000
3,Veh_4,829,1000
4,Veh_5,0,1000
5,Veh_6,0,1000
6,Veh_7,934,1000
7,Veh_8,0,1000
8,Veh_9,0,1000
9,Veh_10,979,1000


In [41]:
# Plot on Map

if 'a' in methods_to_try:
    for veh_ in range(1, number_of_vehicles + 1):
        chosen_color = random.choice(colors_)
        for node_1 in dau_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_i_j_k_a[node_1, node_2, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_a)
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_m_i_k_a[node_1, node_2, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_a)
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_j_m_k_a[node_2, node_1, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_a)

# Display Final Map
map_osm

# b). Miller-Tucker-Zemlin (MTZ) approach

## Solver Inputs

In [42]:
time_limit_b = 600
mip_gap_b = 0.01

## Miller-Tucker-Zemlin (MTZ) approach mathematical formulation

### $ Sets \ : $
#### $ d \ \in \ D: \ Daughter \ Nodes$
#### $ m \ \in \ M: \ Mother \ Nodes$
#### $ k \ \in \ K: \ Total \ available \ vehicles$

### $Parameters:$
#### $ D_{i, \ j}  : \ Distance \ from \ node \ i \ to \ j$
#### $ V_{j} : Demand \ of \ daughter \ node \ j$
#### $ VehCap  : \ Max \ Capacity \ of \ each \ Vehicle$

### $Variables:$
#### $ x_{i, \ j, \ k}  : \ Edge \ selection \ binary \ variable \ between \ two \ daughter \ nodes \ (i \ != \ j) \ via \ vehicle \ k$  
#### $ x_{m, \ i, \ k}  : \ Edge \ selection \ binary \ variable \ between \ mother \ to \ daughter \ node \ via \ vehicle \ k$  
#### $ x_{j, \ m, \ k}  : \ Edge \ selection \ binary \ variable \ between \ daughter \ to \ mother \ node \ via \ vehicle \ k$  
#### $ u_{i, k}  : \ Integer \ variable \ for \ subtour \ elimination$  

### $ Objective \ Function \ : $
### $ min \ \sum_{k\in K} \sum_{m\in M} \sum_{i\in D} \ x_{m, i, k}D_{m, \ i} +
\sum_{k\in K} \sum_{i\in D} \sum_{j\in D} \ x_{i, j, k}D_{i, \ j} +
\sum_{k\in K} \sum_{j\in D} \sum_{m\in M} \ x_{j, m, k}D_{j, \ m}$

### $ Subject \ to \ Constraints: $
#### $1). \ Only \ one \ edge \ entering \ a \ daughter \ node \ can \ be \ active \ (it \ can \ be \ from \ a \ mother \  node \ or \ another \ daughter \ node) $
#### $ \sum_{k\in K} \sum_{m\in M} \ x_{m, i, k} \ + \ \sum_{k\in K} \sum_{j\in D} \ x_{j, i, k} \ = \ 1 \ \forall \ i \in D \$

#### $2). \ At \ any \ node, \ the \ vehicle \ leaving \ should \ be \ the \ same \ as \ vehicle \ entering$
#### $ \ \sum_{m\in M} \ x_{m, i, k} \  + \ \sum_{j\in D} \ x_{j, i, k} \ = \ \sum_{m\in M} \ x_{i, m, k} \  + \ \sum_{j\in D} \ x_{i, j, k} \ \forall \ i \in D \ \forall \ k \in K  $

#### $ \ \sum_{i\in D} \ x_{m, i, k} \ = \ \sum_{j\in D} \ x_{j, m, k} \ \forall \ m \in M \ \forall \ k \in K  $

#### $3). \ If \ a \ vehicle \ is \ being \ used \ to \ service \ atleast \ one \ daughter \ node, \ that \ vehicle \ should \ depart \ from \ a \ mother \ node $

#### $ \ \sum_{i\in D} \ \sum_{j\in D} \ x_{i, j, k} \ <= \ \sum_{m\in M} \ \sum_{j\in D} \ x_{m, j, k} \ * \ M \ \forall \ k \in K  $

#### $ \ \sum_{i\in D} \ \sum_{j\in D} \ x_{i, j, k} \ <= \ z_{k} \ * \ M \ \forall \ k \in K  $

#### $4). A \ vehicle \ can \ depart \ from \ a \ mother \ node \ atmost \ once $

#### $ \ \sum_{m\in M} \ \sum_{j\in D} \ x_{m, j, k} \ <= 1 \ \forall \ k \in K  $

#### $5). \ Each \ vehicle's \ capacity \ constraint $
#### $ \ \sum_{i\in D} \ \sum_{j\in D} \ x_{i, j, k}*V_{j} \ <= \ VehCap \ \forall \ k \in K  $


#### $6). \ Sub-tour \ elimination \ constraint $
#### $ \ u_{j, k} \ - \ u_{i, k} <= \  1 \ - \ M*(1 - x_{i, j, k}) \ \forall \ i \in \ D \ \forall \ j \in \ D \ \forall \ k \in K \$

#### $ \ u_{j, k} \ - \ u_{m, k} <= \  1 \ - \ M*(1 - x_{m, j, k}) \ \forall \ m \in \ M \ \forall \ j \in \ D \ \forall \ k \in K \$

In [43]:
if 'b' in methods_to_try:
    tempo_formulation_b_initial = time.time()
    # Creating Model

    m_b = gp.Model()
    m_b.Params.TIME_LIMIT = time_limit_b
    m_b.Params.MIPGap = mip_gap_b

    # Adding Variables

    x_m_i_k_b = m_b.addVars([(mot, dau, k) for dau in dau_nodes_dict.keys()
                             for mot in mother_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)],
                            vtype=GRB.BINARY, name="mot_to_dau")

    x_i_j_k_b = m_b.addVars([(dau_1, dau_2, k) for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1) if dau_1 != dau_2],
                            vtype=GRB.BINARY, name="dau_to_dau")

    x_j_m_k_b = m_b.addVars([(dau, mot, k) for dau in dau_nodes_dict.keys()
                             for mot in mother_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)],
                            vtype=GRB.BINARY, name="dau_to_mot")
    u_j_k = m_b.addVars([(node_, k) for node_ in all_nodes_info_dict.keys()
                         for k in range(1, number_of_vehicles + 1)],lb=0, vtype=GRB.INTEGER, name='u_variable')

    # Adding Objective Function

    mother_to_daughter_dist_sum = [x_m_i_k_b[mot, dau, k] * distance_matrix[mot,dau] for mot in mother_nodes_dict.keys()
                                           for dau in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)]
    daughter_to_daughter_dist_sum = [x_i_j_k_b[dau_1, dau_2, k] * distance_matrix[dau_1, dau_2] for dau_1 in dau_nodes_dict.keys()
                                           for dau_2 in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1) if dau_1 != dau_2]
    daughter_to_mother_dist_sum = [x_j_m_k_b[dau, mot, k] * distance_matrix[dau, mot] for mot in mother_nodes_dict.keys()
                                           for dau in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)]

    m_b.setObjective(gp.quicksum(mother_to_daughter_dist_sum) + gp.quicksum(daughter_to_daughter_dist_sum)
                       + gp.quicksum(daughter_to_mother_dist_sum), sense=GRB.MINIMIZE)

    # Adding Constraints

    # 1). Only one edge entering a daughter node can be active

    for dau_ent in dau_nodes_dict.keys():
        m_b.addConstr(sum([x_i_j_k_b[dau_1, dau_ent, veh_] for dau_1 in dau_nodes_dict.keys()
                                  for veh_ in range(1, number_of_vehicles + 1) if dau_1 != dau_ent]) + sum(
            [x_m_i_k_b[mot_, dau_ent, veh_] for mot_ in mother_nodes_dict.keys()
             for veh_ in range(1, number_of_vehicles + 1)]) == 1, name=f'dau_enter_edge_active_{dau_ent}')


    # 2). At any node, the vehicle leaving should be the same as vehicle entering

    for dau_ in dau_nodes_dict.keys():
        for veh_ in range(1, number_of_vehicles + 1):
            entering_sum_ = sum([x_i_j_k_b[dau_2, dau_, veh_] for dau_2 in dau_nodes_dict.keys() if dau_ != dau_2]) + sum(
                [x_m_i_k_b[mot_, dau_, veh_] for mot_ in mother_nodes_dict.keys()])
            leaving_sum_ = sum(
                [x_i_j_k_b[dau_, dau_2, veh_] for dau_2 in dau_nodes_dict.keys() if dau_ != dau_2]) + sum(
                [x_j_m_k_b[dau_, mot_, veh_] for mot_ in mother_nodes_dict.keys()])
            m_b.addConstr(entering_sum_ == leaving_sum_ , name=f'veh_dau_enter_leave_{dau_}_{veh_}')

    for mot_ in mother_nodes_dict.keys():
        for veh_ in range(1, number_of_vehicles + 1):
            leaving_sum_ = sum([x_m_i_k_b[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()])
            entering_sum_ = sum([x_j_m_k_b[dau_, mot_, veh_] for dau_ in dau_nodes_dict.keys()])
            m_b.addConstr(entering_sum_ == leaving_sum_, name=f'veh_mot_enter_leave_{mot_}_{veh_}')


    # 3). Every vehicle serving atleast one daughter node has to depart from a mother node

    for veh_ in range(1, number_of_vehicles + 1):
        _entering_sum = sum([x_i_j_k_b[dau_1, dau_2, veh_] for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() if dau_1 != dau_2])
        _leaving_sum = sum([x_m_i_k_b[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()
                            for mot_ in mother_nodes_dict.keys()])
        m_b.addConstr(_entering_sum <= _leaving_sum * 10e3, name=f'vehicle_usage_active_{mot_}')


    # 4). Every vehicle can depart from mother node atmost once

    for veh_ in range(1, number_of_vehicles + 1):
        _leaving_sum = sum([x_m_i_k_b[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()
                            for mot_ in mother_nodes_dict.keys()])
        m_b.addConstr(_leaving_sum <= 1, name=f'veh_dept_only_once_{veh_}')


    # 5). Vehicle capacity constraint

    for veh_ in range(1, number_of_vehicles + 1):
        m_b.addConstr(sum([x_i_j_k_b[dau_1, dau_2, veh_] * dau_nodes_demand[dau_2] for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() if dau_1 != dau_2]) + sum(
            [x_m_i_k_b[mot_, dau_2, veh_] * dau_nodes_demand[dau_2] for mot_ in mother_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys()]) <= cap_per_vehicle, name=f'veh_{veh_}_cap')

    # 6). Subtour elimination constraint

    # For mother node
    for mot_ in mother_nodes_dict.keys():
        for veh_ in range(1, number_of_vehicles + 1):
            m_b.addConstr(u_j_k[mot_, veh_] == 0)

    # For daughter nodes
    for veh_ in range(1, number_of_vehicles + 1):
        for node_1 in dau_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    m_b.addConstr(u_j_k[node_2, veh_] - u_j_k[node_1, veh_] >= 1 - 10e2*(1 - x_i_j_k_b[node_1, node_2, veh_]))

        for mot_ in mother_nodes_dict.keys():
            for node_1 in dau_nodes_dict.keys():
                m_b.addConstr(u_j_k[node_1, veh_] - u_j_k[mot_, veh_] >= 1 - 10e2*(1 - x_m_i_k_b[mot_, node_1, veh_]))

    tempo_solving_b_initial = time.time()
    m_b.optimize()
    objective_b = round(m_b.ObjVal, 2)
    number_of_constraints_b = m_b.numconstrs
    formulation_time_b = round((tempo_solving_b_initial - tempo_formulation_b_initial), 2)
    solving_time_b = round((time.time() - tempo_solving_b_initial), 2)
    print("**************************************************")
    print("Total number of constraints: ", m_b.numconstrs)
    print("Formulation time in seconds for method 'b': ", formulation_time_b)
    print("Solving time in seconds for method 'b':", solving_time_b)
    print("**************************************************")

Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1160 rows, 1210 columns and 8310 nonzeros
Model fingerprint: 0xb53e3bb9
Variable types: 0 continuous, 1210 integer (1100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 10 rows and 290 columns
Presolve time: 0.07s
Presolved: 1150 rows, 920 columns, 6520 nonzeros
Variable types: 0 continuous, 920 integer (820 binary)
Found heuristic solution: objective 7170.4989400

Root relaxation: objective 2.057830e+03, 447 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

## Get Results

In [44]:
# Vehicle usage details

if 'b' in methods_to_try:
    veh_cap_used = {veh_: 0 for veh_ in range(1, number_of_vehicles + 1)}
    dau_nodes_visited = []
    for veh_ in range(1, number_of_vehicles + 1):
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if x_m_i_k_b[node_1, node_2, veh_].X > 0.9:
                    veh_cap_used[veh_] += x_m_i_k_b[node_1, node_2, veh_].X * dau_nodes_demand[node_2]
                    dau_nodes_visited.extend([node_1, node_2])

        for node_1 in dau_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2 and x_i_j_k_b[node_1, node_2, veh_].X > 0.9:
                    veh_cap_used[veh_] += x_i_j_k_b[node_1, node_2, veh_].X * dau_nodes_demand[node_2]
                    dau_nodes_visited.extend([node_1, node_2])

        nodes_visited = set(dau_nodes_visited)
        veh_df = pd.DataFrame({'Vehicle_ID': f'Veh_{k}', 'Volume Captured': int(v),
                               'Vehicle Capacity': cap_per_vehicle} for k, v in veh_cap_used.items())
else:
    veh_df = pd.DataFrame()
veh_df

,Vehicle_ID,Volume Captured,Vehicle Capacity
0,Veh_1,0,1000
1,Veh_2,934,1000
2,Veh_3,975,1000
3,Veh_4,829,1000
4,Veh_5,0,1000
5,Veh_6,0,1000
6,Veh_7,979,1000
7,Veh_8,0,1000
8,Veh_9,0,1000
9,Veh_10,998,1000


In [45]:
# Plot on Map

if 'b' in methods_to_try:
    for veh_ in range(1, number_of_vehicles + 1):
        chosen_color = random.choice(colors_)
        for node_1 in dau_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_i_j_k_b[node_1, node_2, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_b)
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_m_i_k_b[node_1, node_2, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_b)
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_j_m_k_b[node_2, node_1, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_b)

# Display Final Map
map_osm

# c). Lazy constraints approach

## Solver Inputs

In [46]:
time_limit_c = 600
mip_gap_c = 0.01

In [47]:
# Lazy subtour constraint callback functions

def subtourset(edges):
    G = nx.DiGraph(edges)
    simple_cycles = list(nx.simple_cycles(G))
    simple_cycles = [elem + [elem[0]] for elem in simple_cycles]
    return simple_cycles

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        X_i_j_k_c = model.cbGetSolution(x_i_j_k_c)
        for k in range(1, number_of_vehicles + 1):
            edges = [(k_[0], k_[1]) for k_, v_ in X_i_j_k_c.items() if k_[2] == k and v_ > 0.9]
            subtour_list = subtourset(edges)
            if len(subtour_list) > 0:
                for veh_ in range(1, number_of_vehicles + 1):
                    for subtour_ in subtour_list:
                        print("AAdding subtour constraint for: ", subtour_)
                        model.cbLazy(sum([x_i_j_k_c[subtour_[in_], subtour_[in_ + 1], veh_]
                                          for in_ in range(len(subtour_) - 1)]) <= len(subtour_) - 2)

In [48]:
# Creating Model
if 'c' in methods_to_try:

    tempo_formulation_c_initial = time.time()

    m_c = gp.Model()
    m_c.Params.TIME_LIMIT = time_limit_c
    m_c.Params.MIPGap = mip_gap_c
    m_c.Params.lazyConstraints = 1  # Parameter for Lazy Constraints

    # Adding Variables

    x_m_i_k_c = m_c.addVars([(mot, dau, k) for dau in dau_nodes_dict.keys()
                             for mot in mother_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)],
                            vtype=GRB.BINARY, name="mot_to_dau")

    x_i_j_k_c = m_c.addVars([(dau_1, dau_2, k) for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1) if dau_1 != dau_2],
                            vtype=GRB.BINARY, name="dau_to_dau")

    x_j_m_k_c = m_c.addVars([(dau, mot, k) for dau in dau_nodes_dict.keys()
                             for mot in mother_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)],
                            vtype=GRB.BINARY, name="dau_to_mot")

    # Adding Objective Function

    mother_to_daughter_dist_sum = [x_m_i_k_c[mot, dau, k] * distance_matrix[mot,dau] for mot in mother_nodes_dict.keys()
                                           for dau in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)]
    daughter_to_daughter_dist_sum = [x_i_j_k_c[dau_1, dau_2, k] * distance_matrix[dau_1, dau_2] for dau_1 in dau_nodes_dict.keys()
                                           for dau_2 in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1) if dau_1 != dau_2]
    daughter_to_mother_dist_sum = [x_j_m_k_c[dau, mot, k] * distance_matrix[dau, mot] for mot in mother_nodes_dict.keys()
                                           for dau in dau_nodes_dict.keys() for k in range(1, number_of_vehicles + 1)]

    m_c.setObjective(gp.quicksum(mother_to_daughter_dist_sum) + gp.quicksum(daughter_to_daughter_dist_sum)
                       + gp.quicksum(daughter_to_mother_dist_sum), sense=GRB.MINIMIZE)

    # Adding Constraints

    # 1). Only one edge entering a daughter node can be active

    for dau_ent in dau_nodes_dict.keys():
        m_c.addConstr(sum([x_i_j_k_c[dau_1, dau_ent, veh_] for dau_1 in dau_nodes_dict.keys()
                                  for veh_ in range(1, number_of_vehicles + 1) if dau_1 != dau_ent]) + sum(
            [x_m_i_k_c[mot_, dau_ent, veh_] for mot_ in mother_nodes_dict.keys()
             for veh_ in range(1, number_of_vehicles + 1)]) == 1, name=f'dau_enter_edge_active_{dau_ent}')


    # 2). At any node, the vehicle leaving should be the same as vehicle entering

    for dau_ in dau_nodes_dict.keys():
        for veh_ in range(1, number_of_vehicles + 1):
            entering_sum_ = sum([x_i_j_k_c[dau_2, dau_, veh_] for dau_2 in dau_nodes_dict.keys() if dau_ != dau_2]) + sum(
                [x_m_i_k_c[mot_, dau_, veh_] for mot_ in mother_nodes_dict.keys()])
            leaving_sum_ = sum(
                [x_i_j_k_c[dau_, dau_2, veh_] for dau_2 in dau_nodes_dict.keys() if dau_ != dau_2]) + sum(
                [x_j_m_k_c[dau_, mot_, veh_] for mot_ in mother_nodes_dict.keys()])

            m_c.addConstr(entering_sum_ == leaving_sum_ , name=f'veh_dau_enter_leave_{dau_}_{veh_}')

    for mot_ in mother_nodes_dict.keys():
        for veh_ in range(1, number_of_vehicles + 1):
            leaving_sum_ = sum([x_m_i_k_c[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()])
            entering_sum_ = sum([x_j_m_k_c[dau_, mot_, veh_] for dau_ in dau_nodes_dict.keys()])
            m_c.addConstr(entering_sum_ == leaving_sum_, name=f'veh_mot_enter_leave_{mot_}_{veh_}')


    # 3). Every vehicle serving atleast one daughter node has to depart from a mother node

    for veh_ in range(1, number_of_vehicles + 1):
        _entering_sum = sum([x_i_j_k_c[dau_1, dau_2, veh_] for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() if dau_1 != dau_2])
        _leaving_sum = sum([x_m_i_k_c[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()
                            for mot_ in mother_nodes_dict.keys()])
        m_c.addConstr(_entering_sum <= _leaving_sum * 10e3, name=f'vehicle_usage_active_{mot_}')


    # 4). Every vehicle can depart from mother node atmost once

    for veh_ in range(1, number_of_vehicles + 1):
        _leaving_sum = sum([x_m_i_k_c[mot_, dau_, veh_] for dau_ in dau_nodes_dict.keys()
                            for mot_ in mother_nodes_dict.keys()])
        m_c.addConstr(_leaving_sum <= 1, name=f'veh_dept_only_once_{veh_}')


    # 5). Vehicle capacity constraint

    for veh_ in range(1, number_of_vehicles + 1):
        m_c.addConstr(sum([x_i_j_k_c[dau_1, dau_2, veh_] * dau_nodes_demand[dau_2] for dau_1 in dau_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys() if dau_1 != dau_2]) + sum(
            [x_m_i_k_c[mot_, dau_2, veh_] * dau_nodes_demand[dau_2] for mot_ in mother_nodes_dict.keys()
                             for dau_2 in dau_nodes_dict.keys()]) <= cap_per_vehicle, name=f'veh_{veh_}_cap')

    tempo_solving_c_initial = time.time()
    m_c.optimize(subtourelim)
    objective_c = round(m_c.ObjVal, 2)
    number_of_constraints_c = m_c.numconstrs
    formulation_time_c = round((tempo_solving_c_initial - tempo_formulation_c_initial), 2)
    solving_time_c = round((time.time() - tempo_solving_c_initial), 2)
    print("**************************************************")
    print("Total number of constraints: ", m_c.numconstrs)
    print("Formulation time in seconds for method 'c': ", formulation_time_c)
    print("Solving time in seconds for method 'c':", solving_time_c)
    print("**************************************************")

Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 150 rows, 1100 columns and 5300 nonzeros
Model fingerprint: 0xaab4406a
Variable types: 0 continuous, 1100 integer (1100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 5399.6925800
Presolve removed 0 rows and 280 columns
Presolve time: 0.04s
Presolved: 150 rows, 820 columns, 3900 nonzeros
Variable types: 0 continuous, 820 integer (820 binary)

Root relaxation: objective 2.057830e+03, 489 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |    

## Get Results

In [49]:
# Vehicle usage details

if 'c' in methods_to_try:
    veh_cap_used = {veh_: 0 for veh_ in range(1, number_of_vehicles + 1)}
    dau_nodes_visited = []
    for veh_ in range(1, number_of_vehicles + 1):
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if x_m_i_k_c[node_1, node_2, veh_].X > 0.9:
                    veh_cap_used[veh_] += x_m_i_k_c[node_1, node_2, veh_].X * dau_nodes_demand[node_2]
                    dau_nodes_visited.extend([node_1, node_2])

        for node_1 in dau_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2 and x_i_j_k_c[node_1, node_2, veh_].X > 0.9:
                    veh_cap_used[veh_] += x_i_j_k_c[node_1, node_2, veh_].X * dau_nodes_demand[node_2]
                    dau_nodes_visited.extend([node_1, node_2])

        nodes_visited = set(dau_nodes_visited)
        veh_df = pd.DataFrame({'Vehicle_ID': f'Veh_{k}', 'Volume Captured': int(v),
                               'Vehicle Capacity': cap_per_vehicle} for k, v in veh_cap_used.items())
else:
    veh_df = pd.DataFrame()
veh_df

,Vehicle_ID,Volume Captured,Vehicle Capacity
0,Veh_1,0,1000
1,Veh_2,0,1000
2,Veh_3,998,1000
3,Veh_4,933,1000
4,Veh_5,979,1000
5,Veh_6,0,1000
6,Veh_7,0,1000
7,Veh_8,0,1000
8,Veh_9,829,1000
9,Veh_10,975,1000


In [50]:
# Plot on Map

if 'c' in methods_to_try:
    for veh_ in range(1, number_of_vehicles + 1):
        chosen_color = random.choice(colors_)
        for node_1 in dau_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_i_j_k_c[node_1, node_2, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_c)
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_m_i_k_c[node_1, node_2, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_c)
        for node_1 in mother_nodes_dict.keys():
            for node_2 in dau_nodes_dict.keys():
                if node_1 != node_2:
                    if x_j_m_k_c[node_2, node_1, veh_].X > 0.9:
                        points = [(all_nodes_info_dict[node_1][0], all_nodes_info_dict[node_1][1]),
                                  (all_nodes_info_dict[node_2][0], all_nodes_info_dict[node_2][1])]
                        folium.PolyLine(points, color=chosen_color, weight=2, opacity=0.9).add_to(routing_layer_c)

# Display Final Map
map_osm

## Compare the results

In [51]:
summary_dict = {}
if 'a' in methods_to_try:
    new_row_a = {'CVRP Method': 'DFJ Approach', 'Number of Customer Nodes': len(dau_nodes_dict),
                 'Number of Vehicles': number_of_vehicles, 'Objective Value': objective_a,
                 'Number of Constraints': number_of_constraints_a, 'Formulation Time (sec)': formulation_time_a,
                 'Solving Time (sec)': solving_time_a}
    summary_dict['a'] = new_row_a
if 'b' in methods_to_try:
    new_row_b = {'CVRP Method': 'MTZ Approach', 'Number of Customer Nodes': len(dau_nodes_dict),
                 'Number of Vehicles': number_of_vehicles, 'Objective Value': objective_b,
                 'Number of Constraints': number_of_constraints_b, 'Formulation Time (sec)': formulation_time_b,
                 'Solving Time (sec)': solving_time_b}
    summary_dict['b'] = new_row_b
if 'c' in methods_to_try:
    new_row_c = {'CVRP Method': 'Lazy Constraints Approach', 'Number of Customer Nodes': len(dau_nodes_dict),
                 'Number of Vehicles': number_of_vehicles, 'Objective Value': objective_c,
                 'Number of Constraints': number_of_constraints_c, 'Formulation Time (sec)': formulation_time_c,
                 'Solving Time (sec)': solving_time_c}
    summary_dict['c'] = new_row_c

summary_df = pd.DataFrame([val_ for k, val_ in summary_dict.items()])

summary_df

,CVRP Method,Number of Customer Nodes,Number of Vehicles,Objective Value,Number of Constraints,Formulation Time (sec),Solving Time (sec)
0,DFJ Approach,10,10,4500.89,10280,1.04,19.72
1,MTZ Approach,10,10,4500.89,1160,0.07,4.10
2,Lazy Constraints Approach,10,10,4500.89,150,0.04,42.11
